In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The import statement <br>
import_stmt     ::=  "import" module ["as" name] ( "," module ["as" name] )*
                     | "from" relative_module "import" identifier ["as" name]
                     ( "," identifier ["as" name] )*
                     | "from" relative_module "import" "(" identifier ["as" name]
                     ( "," identifier ["as" name] )* [","] ")"
                     | "from" module "import" "*"
module          ::=  (identifier ".")* identifier
relative_module ::=  "."* module | "."+
name            ::=  identifier

The basic import statement (no from clause) is executed in two steps:
find a module, loading and initializing it if necessary
define a name or names in the local namespace for the scope where the import statement occurs.
When the statement contains multiple clauses (separated by commas) the two steps are carried out separately for each clause, just as though the clauses had been separated out into individiual import statements.
The details of the first step, finding and loading modules are described in greater detail in the section on the import system, which also describes the various types of packages and modules that can be imported, as well as all the hooks that can be used to customize the import system. Note that failures in this step may indicate either that the module could not be located, or that an error occurred while initializing the module, which includes execution of the module’s code.
If the requested module is retrieved successfully, it will be made available in the local namespace in one of three ways:

In [ ]:
# importing libraries Kütüphaneleri çağırma/ yükleme
import pandas as pd #Dataframe
import quandl #quandl verileri topladım
import math #matematiksel işlemler
from sklearn import preprocessing, svm #önişlem, destek vektör makineleri çağırdım
from sklearn.linear_model import LinearRegression #linear model veri CSV (excel) 
import numpy as np #matematiksel array
from matplotlib import style #grafikler için
style.use('fivethirtyeight') #bu matplotlkibe ait
from sklearn.model_selection import train_test_split #veriyi bölmek cross_validaition, cross_validation kullanımdan kaldırılmıştır. 

cross_validation is deprecated since version 0.18. This module will be removed in 0.20.

Use sklearn.model_selection.train_test_split instead.

from sklearn.model_selection import train_test_split
<br>
---------------------------------------------------------------------------
ImportError                               Traceback (most recent call last)
/tmp/ipykernel_28/852433834.py in <module>
      3 import quandl
      4 import math
----> 5 from sklearn import preprocessing, cross_validation, svm
      6 from sklearn.linear_model import LinearRegression
      7 import numpy as np

ImportError: cannot import name 'cross_validation' from 'sklearn' (/opt/conda/lib/python3.7/site-packages/sklearn/__init__.py)

In [ ]:
# pip install quandl

In [ ]:
import quandl

In [ ]:
## getting data from quandl
df = quandl.get("WIKI/GOOGL")
# print(df.head())

In [ ]:
df_pd = pd.DataFrame(df)

In [ ]:
df_pd.head(10)

In [ ]:
dfpd.describe().T

In [ ]:
dfpd.columns

In [ ]:
df.keys() #özellikleri görme

In [ ]:
df_pd.shape

In [ ]:
df_pd.size

In [ ]:
# Getting the features we need from the dataframe and dropping the ones we don't need
# Veri dataframe ihtiyacımız olan özellikleri almak ve ihtiyacımız olmayanları atmak

df = df[['Open', 'High', 'Low', 'Close', 'Volume', 'Ex-Dividend', 'Split Ratio','Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume' ]]

In [ ]:
# pca

In [ ]:
# getting the features we need from the data frame and dropping the ones we don't need
# ihtiyacımız olmayan özellikleri atmak

df["HL_PCT"] = (df["High"] - df["Close"]) / df["Close"] * 100.0 #normalization 0-1
df["PCT_change"] = (df["Close"] - df["Open"]) / df["Open"] * 100.0

In [ ]:
df = df[['Close', 'HL_PCT', 'PCT_change', 'Volume']]
print(df.head())

In [ ]:
df_pd.head(4)

In [ ]:
# Defining the label column and the features column and dropping the NaN values
# Etiket sütununu ve özellikler sütununu tanımlama ve NaN değerlerini düşürme

forecast_col = 'Close'  # label column for prediction (y)
df.fillna(-99999, inplace=True)  # fill missing data with outlier value

forecast_out = int(math.ceil(0.01 * len(df)))  # 1% of the length of the dataframe
df['label'] = df[forecast_col].shift(-forecast_out)  # shift the label column up by forecast_out
print(df.head())

In [ ]:
# 0/1
# defining the features and the label and dropping the NaN values from the features
X = np.array(df.drop(['label'], 1))  # features (X)

In [ ]:
X

In [ ]:
df.size- X.size

In [ ]:
print(len(X))

In [ ]:
X = preprocessing.scale(X)  # scale the features
X_lately = X[-forecast_out:]  # features for the last forecast_out rows
X = X[:-forecast_out]  # features for all rows except the last forecast_out rows
df.dropna(inplace=True)  # drop the rows with missing data

In [ ]:
# label(close) test 
y = np.array(df['label'])  # labels (y)
len(y)

In [ ]:
# splitting the data into training and testing data
# veriyi bölme
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=100)

In [ ]:
#model oluştur
#model eğitirken train (X_train ,y_train)

clf = LinearRegression()  # n_jobs=-1 means use all available threads
clf.fit(X_train, y_train) # training the model with the training data  model eğitimi

In [ ]:
accuracy = clf.score(X_test, y_test) # R^2 coefficient of determination, 1 is perfect prediction and 0 is no prediction at all (confidence)
print(f"Doğruluk oranı (Accuracy) %.2f" % accuracy)

In [ ]:
# predicting the stock price for the next forecast_out days

forecast_set = clf.predict(X_lately) # predict the labels for the last forecast_out rows
print(forecast_set)
print()
print(f"Doğruluk oranı (Accuracy) %.2f" % accuracy)
print()
print(forecast_out) # print the predicted labels, accuracy, and forecast_out

df['Forecast'] = np.nan  # create a new column for the predicted labels

**Yeni bir model** <br>support vector regression

In [ ]:
#support vector regression sigmoid
clf = svm.SVR(kernel="sigmoid") # kernel='poly' is the default kernel for SVR and is the best for non-linear data
clf.fit(X_train, y_train) # fit the model to the training data (X_train and y_train) to train the model
accuracy = clf.score(X_test, y_test)    # R^2 coefficient of determination, 1 is perfect prediction and 0 is no prediction at all (confidence)
print(f"accuracy %.2f" % accuracy) # print the accuracy of the model on the test data (X_test and y_test)

In [ ]:
#support vector regression
clf = svm.SVR() # kernel='poly' is the default kernel for SVR and is the best for non-linear data
clf.fit(X_train, y_train) # fit the model to the training data (X_train and y_train) to train the model
accuracy = clf.score(X_test, y_test)    # R^2 coefficient of determination, 1 is perfect prediction and 0 is no prediction at all (confidence)
print(f"accuracy %.2f" % accuracy) # print the accuracy of the model on the test data (X_test and y_test)

In [ ]:
#support vector regression poly
clf = svm.SVR(kernel="poly") # kernel='poly' is the default kernel for SVR and is the best for non-linear data
clf.fit(X_train, y_train) # fit the model to the training data (X_train and y_train) to train the model
accuracy = clf.score(X_test, y_test)    # R^2 coefficient of determination, 1 is perfect prediction and 0 is no prediction at all (confidence)
print(f"accuracy %.2f" % accuracy) # print the accuracy of the model on the test data (X_test and y_test)

In [ ]:
#support vector machine rbf
clf = svm.SVR(kernel='rbf')     # kernel='rbf' is the default kernel for SVM and is the best for non-linear data (default) and linear data
clf.fit(X_train, y_train)      # fit the model to the training data (X_train and y_train) to train the model
accuracy = clf.score(X_test, y_test)   # R^2 coefficient of determination, 1 is perfect prediction and 0 is no prediction at all (confidence)
print(f"accuracy %.2f" % accuracy)  # print the accuracy of the model on the test data (X_test and y_test)

R^score <br>
R², verilerin yerleştirilmiş regresyon hattına ne kadar yakın olduğunun istatistiksel bir ölçüsüdür. Ayrıca belirleme katsayısı veya çoklu regresyon için çoklu belirleme katsayısı olarak da bilinir. Daha basit bir dilde söylemek gerekirse R-kare, doğrusal regresyon modelleri için uygunluk ölçüsüdür.

In [ ]:
#r2 score (coefficient of determination)
y_true = [3, -0.5, 2, 7] 
from sklearn.metrics import r2_score    # import the r2_score from sklearn metrics module
r2_score(y_true, y_pred)   # calculate the r2 score using the actual labels (y_test) and the predicted labels (y_pred)
print(r2_score()) # print the r2 score

> MAE, MSE, RMSE, MAPE [Hakkında](https://forum.yazbel.com/t/mae-mse-rmse-mape-hakkinda/13701)

In [ ]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from seaborn import heatmap
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.neural_network import MLPRegressor
# import matplotlib.style as mplstyle

In [ ]:
#mean absolute error
from sklearn.metrics import mean_absolute_error # import the mean_absolute_error from sklearn metrics module
mean_absolute_error(y_test, y_pred) # calculate the mean absolute error using the actual labels (y_test) and the predicted labels (y_pred)
print(mean_absolute_error) # print the mean absolute error

In [ ]:

#mean squared error
from sklearn.metrics import mean_squared_error # import the mean_squared_error from sklearn metrics module
mean_squared_error(y_test, y_pred) # calculate the mean squared error using the actual labels (y_test) and the predicted labels (y_pred)
print(mean_squared_error) # print the mean squared error

#explained variance score
from sklearn.metrics import explained_variance_score # import the explained_variance_score from sklearn metrics module
explained_variance_score(y_test, y_pred) # calculate the explained variance score using the actual labels (y_test) and the predicted labels (y_pred)
print(explained_variance_score) # print the explained variance score

#mean squared log error
from sklearn.metrics import mean_squared_log_error # import the mean_squared_log_error from sklearn metrics module
mean_squared_log_error(y_test, y_pred) # calculate the mean squared log error using the actual labels (y_test) and the predicted labels (y_pred)
print(mean_squared_log_error) # print the mean squared log error

#median absolute error
from sklearn.metrics import median_absolute_error # import the median_absolute_error from sklearn metrics module
median_absolute_error(y_test, y_pred) # calculate the median absolute error using the actual labels (y_test) and the predicted labels (y_pred)
print(median_absolute_error) # print the median absolute error

#mean absolute percentage error
from sklearn.metrics import mean_absolute_percentage_error # import the mean_absolute_percentage_error from sklearn metrics module
mean_absolute_percentage_error(y_test, y_pred) # calculate the mean absolute percentage error using the actual labels (y_test) and the predicted labels (y_pred)
print(mean_absolute_percentage_error) # print the mean absolute percentage error

In [ ]:
#support vector machine (linear) classifier 
clf = LinearRegression(n_jobs=-1)
# create the classifier object using the LinearRegression class
clf.fit(X_train, y_train) # fit the model to the training data (X_train and y_train) to train the model
accuracy = clf.score(X_test, y_test) # calculate the accuracy of the model using the test data (X_test and y_test)
print(f"accuracy %.2f"% accuracy) # print the accuracy

In [ ]:
forecast_set = clf.predict(X_lately) # predict the labels for the test data (X_lately) using the trained model
print(forecast_set, accuracy, forecast_out) # print the predicted labels (forecast_set), the accuracy (accuracy), and the forecast_out (forecast_out)


In [ ]:
import datetime

In [ ]:
last_date = df.iloc[-1].name # get the last date from the dataframe
last_unix = last_date.timestamp() # get the timestamp of the last date
one_day = 86400 # set the number of seconds in a day
next_unix = last_unix + one_day # get the timestamp of the next day

for i in forecast_set: # iterate through the forecast_set
    next_date = datetime.datetime.fromtimestamp(next_unix) # get the date from the timestamp
    next_unix += 86400 # add 86400 seconds to the timestamp
    df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)] + [i] # add the predicted label (i) to the dataframe

figure = plt.figure(figsize=(10, 10))
df['Close'].plot() # plot the Adj. Close column of the dataframe as a line plot
df['Forecast'].plot() # plot the Forecast column of the dataframe as a line plot
plt.legend(loc=4) # set the legend of the plot
plt.xlabel('Date') # set the x-axis label of the plot
plt.ylabel('Price') # set the y-axis label of the plot
plt.show() # show the plot


In [ ]:
#saving The model
import pickle #importing picke module
with open('linearregression.pickle','wb') as LRModel:
    pickle.dump(clf,LRModel)
    print("saved")

In [ ]:
#load the model
pickle_in = open('linearregression.pickle','rb') # open the file linearregression.pickle in read binary mode
clf = pickle.load(pickle_in) # load the classifier from the file
accuracy = clf.score(X_test,y_test)
print(f"accuracy %.2f" %accuracy)

Follow on [Github](https://github.com/themanoftalent)